In [1]:
import utilities as u
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader

In [13]:
class TopLevelProofDataset(InMemoryDataset):
    def __init__(self, root='', transform=None, pre_transform=None):
        super(TopLevelProofDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['../top_level_proofs_0.dataset']
    
    def download(self):
        pass
    
    def process(self):
        data_list = []
        
        for thm, y in datapoints:
            thm = u.process_theorem(thm)
            tree, distinct_features = u.thm_to_tree(thm)
            
            x, edge_index = u.graph_to_data(tree, list(distinct_features))
            
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
test_thm = '(fun (a A B) (a A (a A B)))'
print(test_thm)
thm = u.process_theorem(test_thm)
print(thm)
thm_tree = u.thm_to_tree(thm)
print(thm_tree)

(fun (a A B) (a A (a A B)))
['(', 'fun', '(', 'a', 'A', 'B', ')', '(', 'a', 'A', '(', 'a', 'A', 'B', ')', ')', ')']
(<utilities.data_structures.Tree object at 0x7f0a2c655510>, {'A', 'a', 'B', 'fun'})


In [6]:
datapoints = u.make_data()
datapoints = datapoints[0:2]

0


In [7]:
for idx, (thm, _) in enumerate(datapoints):
    if idx % 1000 == 0:
        print(f'{idx} / {len(datapoints)}')
    thm = u.process_theorem(thm)

0 / 2


In [14]:
dataset = TopLevelProofDataset()
print(dataset)
loader = DataLoader(dataset, batch_size=1, shuffle=True)
for batch in loader:
    print(batch)

Processing...
Done!
TopLevelProofDataset(2)
Batch(batch=[43], edge_index=[2, 84], x=[43, 1], y=[1])
Batch(batch=[115], edge_index=[2, 228], x=[115, 1], y=[1])


In [15]:
counter = dict()
for _, y in datapoints:
    if y in counter:
        counter[y] += 1
    else:
        counter[y] = 1
counter = list(counter.items())
counter.sort(key=lambda x: x[1], reverse=True)
percentages = [(x, y/len(datapoints)*100) for x,y in counter]
percentages

[(1, 50.0), (2, 50.0)]